<a href="https://colab.research.google.com/github/KellySantosG/Progrmaci-n-tareas-/blob/main/Proyecto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Kelly Santos García
#Aqui está lo que importamos
import numpy as np
import pandas as pd
from google.colab import drive

#Aquí empieza el codigo tal cual
class Grafica:
    """Clase base para representar una gráfica."""
    def __init__(self, nodos, arcos):
        self.matriz = np.zeros((len(nodos), len(nodos)))
        for x in arcos:
            self.matriz[x[0], x[1]] = 1

    def __str__(self):
        return str(self.matriz)

#Aquí implementamos la clase Red
class Red(Grafica):
    """Clase Red que hereda de Grafica y calcula PageRank."""
    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)
        self.matriz_probabilidades = np.zeros_like(self.matriz)

    def calcular_matriz_probabilidades(self):
        """Normaliza los arcos para generar una matriz de transición probabilística."""
        for i in range(len(self.matriz)):
            suma = self.matriz[i, :].sum()
            if suma > 0:
                self.matriz_probabilidades[i, :] = self.matriz[i, :] / suma
            else:
                self.matriz_probabilidades[i, :] = np.zeros(len(self.matriz))

    def calcular_pagerank(self, d=0.85, tol=1e-6):
        """Calcula el PageRank utilizando el factor de amortiguación."""
        n = len(self.matriz)
        self.calcular_matriz_probabilidades()
        U = np.ones((n, n)) / n
        P = d * self.matriz_probabilidades + (1 - d) * U
        pi = np.ones((1, n)) / n
        pi_1 = pi @ P

        while np.linalg.norm(pi - pi_1) > tol:
            pi = pi_1
            pi_1 = pi @ P

        return pi_1.flatten()

    def calcular_pagerank_ru(self, indices_ru, d=0.85, tol=1e-6):
        """
        Calcula el PageRank con un vector inicial modificado para considerar solo páginas .ru.
        """
        n = len(self.matriz)
        s = len(indices_ru)
        pi = np.zeros((1, n))
        pi[0, indices_ru] = 1 / s  # Asignar pesos uniformes a las páginas .ru

        self.calcular_matriz_probabilidades()
        U = np.ones((n, n)) / n
        P = d * self.matriz_probabilidades + (1 - d) * U
        pi_1 = pi @ P

        while np.linalg.norm(pi - pi_1) > tol:
            pi = pi_1
            pi_1 = pi @ P

        return pi_1.flatten()

def construccion_red(ruta_nombre_del_archivo):
    """
    Construye un objeto Red a partir de un archivo Excel.
    """
    df = pd.read_excel(ruta_nombre_del_archivo)
    Nodos = list(range(len(df["Index"])))
    Arcos = []

    for i, citados in enumerate(df["Cited by"]):
        for citado in str(citados).split(','):
            if citado.strip().isdigit():
                Arcos.append((int(citado.strip()) - 1, i))

    return Red(Nodos, Arcos), df

#Aquí implementamos el main requerido
if __name__ == "__main__":
    # Montar Google Drive
    drive.mount('/content/drive', force_remount=True)
    ruta_excel = "/content/drive/MyDrive/Web.xlsx"

    # Construir la red desde el archivo Excel
    red, df = construccion_red(ruta_excel)

    #Empezamos con la zona de lo de las preguntas
    # Pregunta 1: Calcular el PageRank estándar
    pagerank_resultado = red.calcular_pagerank()
    pagina_importante = np.argmax(pagerank_resultado) + 1  # Sumar 1 porque los índices inician en 0

    print("Resultados de PageRank estándar (Pregunta 1):", pagerank_resultado)
    print("Página más importante (Pregunta 1):", pagina_importante)

    # Pregunta 2: Identificar páginas con terminación .ru
    if "Website" in df.columns:
        indices_ru = [i for i, url in enumerate(df["Website"]) if str(url).endswith('.ru')]

        if indices_ru:
            pagerank_ru_resultado = red.calcular_pagerank_ru(indices_ru)
            pagina_importante_ru = np.argmax(pagerank_ru_resultado) + 1

            print("\nResultados de PageRank con páginas .ru (Pregunta 2):", pagerank_ru_resultado)
            print("Página más importante con .ru (Pregunta 2):", pagina_importante_ru)
        else:
            print("\nNo hay páginas con terminación .ru en los datos proporcionados (Pregunta 2).")
    else:
        print("\nLa columna 'Website' no existe en los datos (Pregunta 2).")

    # Pregunta 3: Calcular PageRank con diferentes valores de d
    valores_d = [0.5, 0.85, 1.0]
    print("\nResultados para diferentes valores de d (Pregunta 3):")
    for d in valores_d:
        pagerank_modificado = red.calcular_pagerank(d=d)
        print(f"\nd={d}")
        print("Vector de PageRank:", pagerank_modificado)
        print("Página más importante:", np.argmax(pagerank_modificado) + 1)


Mounted at /content/drive
Resultados de PageRank estándar (Pregunta 1): [4.36095886e-06 5.71216390e-06 4.15619363e-06 7.63861952e-06
 3.75063521e-06 4.93162098e-06 7.20351852e-06 5.05091425e-06
 6.13151513e-06 5.29650322e-06 3.45797359e-06 3.61858330e-06
 4.68195088e-06 4.60022938e-06 5.85797202e-06 7.17214702e-06
 7.22936600e-06 7.15264950e-06 7.18266617e-06 9.27941187e-06
 7.18266617e-06 7.14311332e-06 7.08846838e-06 9.28740993e-06
 7.12996108e-06 5.55837256e-06]
Página más importante (Pregunta 1): 24

Resultados de PageRank con páginas .ru (Pregunta 2): [4.40413124e-06 5.76871288e-06 4.19733890e-06 7.71423992e-06
 3.78776555e-06 4.98044278e-06 7.27483153e-06 5.10091702e-06
 6.19221558e-06 5.34893726e-06 3.49220665e-06 3.65440636e-06
 4.72830101e-06 4.64577049e-06 5.91596446e-06 7.24314946e-06
 7.30093489e-06 7.22345892e-06 7.25377275e-06 9.37127569e-06
 7.25377275e-06 7.21382833e-06 7.15864242e-06 9.37935294e-06
 7.20054589e-06 5.61339904e-06]
Página más importante con .ru (Pregunta